<a href="https://colab.research.google.com/github/arturoac2000/Inteligencia-Artifical/blob/main/Proyecto3(168724).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from collections import Counter
from random import choices
from tqdm import tqdm

In [2]:
tiempo_max = 72*60

names = ["Madrid", "Barcelona", "Paris", "Lyon", "London", "Brussels",
                "Frankfurt", "Milan", "Amsterdam", "Cologne", "Berlin", "Roma"]

adyacencia = np.asarray([
    #0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11
    [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], #Madrid
    [1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0], #Barcelona
    [1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0], #Paris
    [0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0], #Lyon
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], #London
    [0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0], #Brussels
    [0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0], #Frankfurt
    [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1], #Milan
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0], #Amsterdam
    [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0], #Cologne
    [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0], #Berlin
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], #Roma
])

costo_t = np.asarray([
    [0, 150, 225, 0, 0, 0, 0, 0, 0, 0, 0, 0], #Madrid
    [150, 0, 390, 200, 0, 0, 0, 0, 0, 0, 0, 0], #Barcelona
    [225, 390, 0, 112, 136, 82, 480, 0, 0, 0, 0, 0], #Paris
    [0, 200, 112, 0, 0, 0, 0, 176, 0, 0, 0, 0], #Lyon
    [0, 0, 136, 0, 0, 0, 0, 0, 0, 0, 0, 0], #London
    [0, 0, 82, 0, 0, 0, 0, 0, 120, 0, 0, 0], #Brussels
    [0, 0, 480, 0, 0, 0, 0, 454, 0, 120, 232, 0], #Frankfurt
    [0, 0, 0, 176, 0, 0, 454, 0, 0, 0, 0, 168], #Milan
    [0, 0, 0, 0, 0, 120, 0, 0, 0, 120, 364, 0], #Amsterdam
    [0, 0, 0, 0, 0, 0, 120, 0, 120, 0, 0, 0], #Cologne
    [0, 0, 0, 0, 0, 0, 232, 0, 364, 0, 0, 0], #Berlin
    [0, 0, 0, 0, 0, 0, 0, 168, 0, 0, 0, 0], #Roma
])

costo_d = np.asarray([
    [0, 98, 380, 0, 0, 0, 0, 0, 0, 0, 0, 0], #Madrid
    [98, 0, 400, 320, 0, 0, 0, 0, 0, 0, 0, 0], #Barcelona
    [380, 400, 0, 185, 98, 80, 345, 0, 0, 0, 0, 0], #Paris
    [0, 320, 185, 0, 0, 0, 0, 180, 0, 0, 0, 0], #Lyon
    [0, 0, 98, 0, 0, 0, 0, 0, 0, 0, 0, 0], #London
    [0, 0, 80, 0, 0, 0, 0, 0, 48, 0, 0, 0], #Brussels
    [0, 0, 345, 0, 0, 0, 0, 240, 0, 40, 125, 0], #Frankfurt
    [0, 0, 0, 180, 0, 0, 240, 0, 0, 0, 0, 125], #Milan
    [0, 0, 0, 0, 0, 48, 0, 0, 0, 40, 235, 0], #Amsterdam
    [0, 0, 0, 0, 0, 0, 40, 0, 40, 0, 0, 0], #Cologne
    [0, 0, 0, 0, 0, 0, 125, 0, 235, 0, 0, 0], #Berlin
    [0, 0, 0, 0, 0, 0, 0, 125, 0, 0, 0, 0], #Roma
])

In [3]:
def create_individual(gen_size, city_no):
    individual = []
    individual.append(int(np.random.randint(city_no + 1)))
    for _ in range(gen_size):
        next_city = int(np.random.randint(0, city_no))
        while not adyacencia[individual[-1], next_city]:
            next_city = int(np.random.randint(0, city_no))
        individual.append(next_city)
        if len(set(individual)) == city_no + 1:
            break
    return individual

In [4]:
def initial_population(pop_size, gen_size, city_no):
    return [create_individual(gen_size, city_no) for _ in range(pop_size)]

In [5]:
def crossover(parent1, parent2):
    common_cities = set(parent1).intersection(parent2)
    if not common_cities:
        return create_individual(len(parent1), len(adyacencia) - 1), create_individual(len(parent2), len(adyacencia) - 1)

    city = np.random.choice(list(common_cities))
    index_parent1, index_parent2 = parent1.index(city), parent2.index(city)
    return parent1[:index_parent1] + parent2[index_parent2:], parent2[:index_parent2] + parent1[index_parent1:]

In [6]:
def create_new_population(population, pop_size, elitism_coef):
    sorted_population = sorted(population, key=lambda ind: fitness(ind), reverse=True)
    fitness_values = [fitness(ind) for ind in sorted_population]
    if sum(fitness_values) == 0:
        return initial_population(pop_size, len(sorted_population[0]), len(adyacencia) - 1)

    normalized_fitness = [fit / sum(fitness_values) for fit in fitness_values]
    new_population = sorted_population[:elitism_coef]

    for _ in range((pop_size - elitism_coef) // 2):
        parents = choices(sorted_population, weights=normalized_fitness, k=2)
        children = crossover(parents[0], parents[1])
        new_population.extend(children)

    return new_population

In [7]:
def fitness(individual):
    total_cost, total_time = 0, 0
    for i in range(len(individual) - 1):
        total_cost += costo_d[individual[i]][individual[i + 1]]
        total_time += costo_t[individual[i]][individual[i + 1]]
    return 0 if total_time > tiempo_max else total_cost

In [8]:
def describe_route(individual):
    route_str = "Ruta: "
    for i in range(len(individual) - 1):
        route_str += f"{names[individual[i]]} -> "
    return route_str + f"{names[individual[-1]]}"

In [9]:
def cities(population):
    """
    Esta función toma una lista de genomas y devuelve una lista con el número de ciudades únicas visitadas por cada genoma.
    """
    unique_cities = []
    for genome in population:
        unique_cities.append(len(set(genome)))
    return unique_cities

In [10]:
def pop_fitness(population):
    """
    Esta función toma una población de genomas y devuelve una lista de las aptitudes (fitness) de cada genoma.
    """
    fitness_values = []
    for genome in population:
        fitness_values.append(fitness(genome))
    return fitness_values

In [11]:
def solution(genome, city_names):
    """
    Esta función toma un genoma y una lista de nombres de ciudades y devuelve
    una representación legible de la solución, que es una secuencia de ciudades.
    """
    city_sequence = []
    for city in genome:
        city_sequence.append(city_names[city])
    return ' -> '.join(city_sequence)

In [12]:
def create_genome_v2(gen_size):
    genome = []
    genome.append(int(np.random.randint(city_no + 1)))
    for _ in range(gen_size):
        next_city = int(np.random.randint(0, city_no))
        while adyacencia[genome[-1], next_city] == 0:
            next_city = int(np.random.randint(0, city_no))
        genome.append(next_city)
        if cities([genome])[0] == city_no + 1:
            break
    return genome

In [13]:
def mutate_v2(genome1, genome2):
    common_cities = set(genome1).intersection(genome2)
    if not common_cities:
        child1 = create_genome_v2(len(genome1))
        child2 = create_genome_v2(len(genome2))
        return child1, child2
    city = np.random.choice(list(common_cities))
    p = genome1.index(city)
    q = genome2.index(city)
    child1 = genome1[:p] + genome2[q:]
    child2 = genome2[:q] + genome1[p:]
    return child1, child2

In [14]:
def next_generation_v2(pop, pop_size, elitism_coef):
    fit = pop_fitness(pop)
    if np.all(fit == 0) or sum(fit) == 0:
        return [create_genome_v2(gen_size) for _ in range(pop_size)]

    cities_no = cities(pop)
    temp = pop.copy()
    temp.sort(key=fitness, reverse=True)
    for i in range(len(fit)):
        if fit[i] != 0:
            fit[i] += cities_no[i]

    next_gen = temp[:elitism_coef]

    for _ in range(int(np.floor((pop_size - elitism_coef) / 2))):
        pair = choices(pop, weights=fit, k=2)
        next_gen.extend(mutate_v2(pair[0], pair[1]))

    return next_gen

In [15]:
# Variables globales y configuración

city_no = len(adyacencia) - 1
pop_size = 40
gen_size = 60
epochs = 200
elitism_coef = 2

In [16]:
# Algoritmo genético

while True:
    pop = [create_genome_v2(gen_size) for _ in range(pop_size)]
    for _ in tqdm(range(epochs)):
        pop = next_generation_v2(pop, pop_size, elitism_coef)

    pop.sort(key=fitness, reverse=True)
    fit = pop_fitness(pop)
    if cities(pop)[0] == city_no + 1:
        break

100%|██████████| 200/200 [00:01<00:00, 137.25it/s]


In [17]:
# Encontrar la mejor solución
best_genome = min(pop, key=fitness)
print("Mejor ruta encontrada:", solution(best_genome, names))
print("Número de ciudades únicas visitadas en la mejor ruta:", cities(pop)[0])
print("Costo total: ", fitness(best_genome))
for gene in pop:
    print(gene)

Mejor ruta encontrada: Roma -> Milan -> Lyon -> Milan -> Frankfurt -> Berlin -> Frankfurt -> Cologne -> Amsterdam -> Brussels -> Amsterdam -> Brussels -> Paris -> London -> Paris -> London -> Paris -> Madrid -> Barcelona
Número de ciudades únicas visitadas en la mejor ruta: 12
Costo total:  2149
[11, 7, 3, 7, 6, 10, 6, 9, 8, 5, 8, 5, 2, 4, 2, 4, 2, 0, 1]
[11, 7, 3, 7, 6, 10, 6, 9, 8, 5, 8, 5, 2, 4, 2, 4, 2, 0, 1]
[11, 7, 3, 7, 6, 10, 6, 9, 8, 5, 8, 5, 2, 4, 2, 4, 2, 0, 1]
[11, 7, 3, 7, 6, 10, 6, 9, 8, 5, 8, 5, 2, 4, 2, 4, 2, 0, 1]
[11, 7, 3, 7, 6, 10, 6, 9, 8, 5, 8, 5, 2, 4, 2, 4, 2, 0, 1]
[11, 7, 3, 7, 6, 10, 6, 9, 8, 5, 8, 5, 2, 4, 2, 4, 2, 0, 1]
[11, 7, 3, 7, 6, 10, 6, 9, 8, 5, 8, 5, 2, 4, 2, 4, 2, 0, 1]
[11, 7, 3, 7, 6, 10, 6, 9, 8, 5, 8, 5, 2, 4, 2, 4, 2, 0, 1]
[11, 7, 3, 7, 6, 10, 6, 9, 8, 5, 8, 5, 2, 4, 2, 4, 2, 0, 1]
[11, 7, 3, 7, 6, 10, 6, 9, 8, 5, 8, 5, 2, 4, 2, 4, 2, 0, 1]
[11, 7, 3, 7, 6, 10, 6, 9, 8, 5, 8, 5, 2, 4, 2, 4, 2, 0, 1]
[11, 7, 3, 7, 6, 10, 6, 9, 8, 5, 8, 5, 2, 4